### Description:

In this task you will analyse software T-table AES-128 implementation running on ESP32 (see the code_sample.txt in the support folder).  

The software implementation does not contain any protection against side-channel attacks.

During one AES-128 execution a plaintext, a ciphertext and a real power trace of 5,000 samples were recorded.

10,000 encryptions were performed. Associated data (a plaintext, a ciphertext and a trace) was stored in HDF5 file.

One oscilloscope snapshot shows an entire AES execution, however, only the part related to encryption process, i.e., a part shown from 50 us to 60 us on the oscilloscope snapshot, is available for the analysis. 

The measurements were performed with LeCroy WaveRunner Zi625 and a differential probe.

### Tips:

A 'trace' is a side-channel information measured between the Vdd and the GND pins of the WROOM module (ESP32). 

Usually, CPU is vulnerable either to Hamming weight or to Hamming distance data models. There might be more than one instruction leaking secret data.

In this example a trace is a 5,000 bytes array of int8 (the amplitude of the measured voltage).

This array contains various encryption information measured with noise.

### Task:

You might perform several subtasks, but the main objective is to get the Master Key.

The master key is in the form of SCA{XXXXXXXXXXX}, where X is an ASCII printable symbol.

### AES-128 encryption measured at high sampling rate

Green signal - a GPIO trigger used to synchronize all the traces

Red signal   - VCC voltage variation during AES-128 encryption

<img src="support/osc_snap.png">

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import string
import binascii
import h5py
import numpy as np
import matplotlib.pyplot as plt
import numpy.matlib
import subprocess
import sca_training

%matplotlib inline

datapath        = r'../data' 

hdffilename_data    = datapath + r'/attack_data.hdf5'
hdffilename_traces  = datapath + r'/attack_traces.hdf5'

hdf5_dataset          = 'Traces'

numtraces   = 10000
numsamples  = 5000

#----------------------------------------------------------------------------
# OUTPUTS:
#     ptexts   - input plaintexts converted to numpy array of uint8
#     ctexts   - resulted ciphertext converted to numpy array of uint8
#     traces   - a trace associated with encryption process
#----------------------------------------------------------------------------

def ReadData():
    print('Reading data...')
    with h5py.File(hdffilename_data, 'r') as hdf5_data:      
        ptexts = hdf5_data['Ptexts'][:,:].astype('uint8')
        ctexts = hdf5_data['Ctexts'][:,:].astype('uint8')

    with h5py.File(hdffilename_traces, 'r') as hdf5_traces:      
        traces = hdf5_traces[hdf5_dataset][:,:].astype('int8')

    print('Ptexts size:', ptexts.shape)
    print('Ctexts size:', ctexts.shape)
    print('Traces size:', traces.shape)
    print('Completed')
    
    return ptexts, ctexts, traces

ptexts, ctexts, traces = ReadData()

Reading data...
Ptexts size: (10000, 16)
Ctexts size: (10000, 16)
Traces size: (10000, 5000)
Completed


### Task 0
Plot the mean for the first 1000 traces.

Do you see patterns?

Can you tell more about AES-128 operations?

### Task 1
Compute correlation with plaintexts and ciphertexts. There are few associated questions:
    1. Do you consider correlation to be strong?
    2. When is the correlation time?
    3. Does the same model correlate only once?
    4. Does different models correlate differently?
    5. How can this correlation help us to find the key?

### Task 2

Create a model that involve any of the round keys. Use this model to find the round key. Prove that the key is correct.